# **COVID-19 TASK - 1 transmission incubation environment 2.0**

**REVISION - 2020-03-30 code cleaned, modularized and commented**

![](https://sportslogohistory.com/wp-content/uploads/2018/09/georgia_tech_yellow_jackets_1991-pres-1.png)

**PROBLEM:** When a new virus is discovered and causes a pandemic, it is important for scientists to get information coming from all scientific sources that may help them combat the pandemic.  The challenege, however, is that the number of scientific papers created is large and the papers are published very rapidly, making it nearly impossible for scientists to digest and understand important data in this mass of data.

**SOLUTION:** Create an unsupervised scientific literature understanding system that can take in common terms and analyze a very large corpus of scientific papers and return highly relevant text excerpts from papers containing topical data relating to the common text inputed, allowing a single researcher or small team to gather targeted information and quickly and easily locate relevant text in the scientific papers to answer important questions about the new virus from a large corpus of documents.

**APPROACH:** The newest implementation outlined:
- open the meta CSV file in a dataframe - 44K entries
- drop the duplicates by abstract
- loop (comprehension) a list of lists for the keyword for searches. eg. search=[['incubation','period','range']]
- stem the keywords so incubation becomes incubat etc.
- append indpendently - different variations of corona references - (covid, cov, -cov, hcov) to the keywords avoids pulling older research
- query function in pandas to query the above terms with the keywords - the abstract has to have all the keywords and at least one of (covid, cov, -cov, hcov) 
- drop the duplicates in the df again after the queries
- caclulate a relevance score for all the abstracts in the query result df
- raw_score = total count of the keywords in the abstract
- final_score = (raw_score/len(abstract))*raw_score
- sort the df on the final_score ascending=False (best on top)
- drop rows with a relevance score < .02
- parse the relevant abstarcts into sentences split('. ') works well.
- test if keywords are in the sentences
- if sentence has all keywords, it is added to the df for display.
- if you are seeking statistics in the data, adding % to the search terms works well
- df with search results displayed in HTML - currently limiting results to 3 for ease of scanning

**Pros:** Currently the system is a very simple **(as Einstein said "make it as simple as possible, but no simpler")**, but quite effective solutiuon to providing insight on topical queries.  The code is easy to read and simple to understand.

**Cons:** Currently the system requires some human understanding in crafting keyword combinations, however, the recent relevance measure changes and stemming of keywords, have made it so the keywords can be very close to the NL questions and return very good results.


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# load the meta data from the CSV file using 3 columns (abstract, title, authors),
df=pd.read_csv('/kaggle/input/CORD-19-research-challenge/metadata.csv', usecols=['title','abstract','authors','doi','publish_time'])
print (df.shape)
#drop duplicate abstracts
df = df.drop_duplicates(subset='title', keep="first")
#drop NANs 
df=df.dropna()
# convert abstracts to lowercase
df["abstract"] = df["abstract"].str.lower()
#show 5 lines of the new dataframe
print (df.shape)
df.head()

(45774, 5)
(34776, 5)


,title,doi,abstract,publish_time,authors
0,SIANN: Strain Identification by Alignment to N...,10.1101/001727,next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...
1,Spatial epidemiology of networked metapopulati...,10.1101/003889,an emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li
2,Sequencing of the human IG light chain loci fr...,10.1101/006866,germline variation at immunoglobulin gene (ig)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...
3,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol
4,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen


In [2]:
import functools
from IPython.core.display import display, HTML
from nltk import PorterStemmer

# function to stem keyword list into a common base word
def stem_words(words):
    stemmer = PorterStemmer()
    singles=[]
    for w in words:
        singles.append(stemmer.stem(w))
    return singles

def search_dataframe(df,search_words):
    search_words=stem_words(search_words)
    anchor_words=['covid','-cov-2','hcov']
    # add cov to focus the search the papers and avoid unrelated documents
    df_name=['dfa','dfb','dfc']
    z=0
    for anchor in anchor_words:
        search_words.append(anchor)
        # search the dataframe for all the keywords
        df_name[z]=df[functools.reduce(lambda a, b: a&b, (df['abstract'].str.contains(s) for s in search_words))]
        search_words.pop()
        z=z+1
    #combine frames with COVID and cov and corona and drop dups
    frames = [df_name[0], df_name[1], df_name[2]]
    df1 = pd.concat(frames)
    df1=df1.drop_duplicates(subset='title', keep="first")
    return df1

# function analyze search results for relevance with word count / abstract length
def search_relevance(rel_df,search_words):
    rel_df['score']=""
    search_words=stem_words(search_words)
    for index, row in rel_df.iterrows():
        abstract = row['abstract']
        result = abstract.split()
        len_abstract=len(result)
        score=0
        for word in search_words:
            score=score+result.count(word)
        final_score=(score/len_abstract)
        rel_score=score*final_score
        rel_df.loc[index, 'score'] = rel_score
    rel_df=rel_df.sort_values(by=['score'], ascending=False)
    #rel_df= rel_df[rel_df['score'] > .01]
    return rel_df

# function to get best sentences from the search results
def get_sentences(df1,search_words):
    df_table = pd.DataFrame(columns = ["pub_date","authors","title","excerpt","rel_score"])
    search_words=stem_words(search_words)
    for index, row in df1.iterrows():
        pub_sentence=''
        sentences_used=0
        #break apart the absracrt to sentence level
        sentences = row['abstract'].split('. ')
        #loop through the sentences of the abstract
        highligts=[]
        for sentence in sentences:
            # missing lets the system know if all the words are in the sentence
            missing=0
            #loop through the words of sentence
            for word in search_words:
                #if keyword missing change missing variable
                if word not in sentence:
                    missing=1
                #if '%' in sentence:
                    #missing=missing-1
            # after all sentences processed show the sentences not missing keywords
            if missing==0 and len(sentence)<1000 and sentence!='':
                sentence=sentence.capitalize()
                if sentence[len(sentence)-1]!='.':
                    sentence=sentence+'.'
                pub_sentence=pub_sentence+'<br><br>'+sentence
        if pub_sentence!='':
            sentence=pub_sentence
            sentences_used=sentences_used+1
            authors=row["authors"].split(" ")
            link=row['doi']
            title=row["title"]
            score=row["score"]
            linka='https://doi.org/'+link
            linkb=title
            sentence='<p fontsize=tiny" align="left">'+sentence+'</p>'
            final_link='<p align="left"><a href="{}">{}</a></p>'.format(linka,linkb)
            to_append = [row['publish_time'],authors[0]+' et al.',final_link,sentence,score]
            df_length = len(df_table)
            df_table.loc[df_length] = to_append
    return df_table
    
################ main program ########################

display(HTML('<h1>Task 1: What is known about transmission, incubation, and environmental stability?</h1>'))

display(HTML('<h3>Results currently limited to two (2) for ease of scanning</h3>'))

# list of lists of search terms
questions=[
['Q: What is the range of incubation periods for the disease in humans?'],
['Q: How long are individuals are contagious?'],
['Q: How long are individuals are contagious, even after recovery.'],
['Q: Does the range of incubation period vary across age groups?'],
['Q: Does the range of incubation period vary with children?'],
['Q: Does the range of incubation period vary based on underlying health?'],
['Q: What do we know about the basic reproduction number?'],
['Q: What is the prevalance of asymptomatic transmission?'],
['Q: What do we know about asymptomatic transmission in children?'],
['Q: What do we know about seasonality of transmission?'],
['Q: Informing decontamination based on physical science of the coronavirus?'],
['Q: What do we know about stability of the virus in environmental conditions?'],
['Q: What do we know about persistence of the virus on various substrates? (e.g., nasal discharge, sputum, urine, fecal matter, blood)'],
['Q: What do we know about persistence of the virus on various surfaces? (e,g., copper, stainless steel, plastic) '],
['Q: What do we know about viral shedding duration?'],
['Q: What do we know about viral shedding in fecal/stool?'],
['Q: What do we know about viral shedding from nasopharynx?'],
['Q: What do we know about viral shedding in blood?'],
['Q: What do we know about viral shedding in urine?'],
['Q: What do we know about implemtation of diagnostics?'],
['Q: What do we know about disease models?'],
['Q: What do we know about models for disease infection?'],
['Q: What do we know about models for disease transmission?'],
['Q: Are there studies about phenotypic change?'],
['Q: What is know about adaptations (mutations) of the virus?'],
['Q: What do we know about the human immune response and immunity?'],
['Q: Is population movement control effective in stopping transmission (spread)?'],
['Q: Effectiveness of personal protective equipment (PPE)?'],
['Q: What is the role of environment in transmission?']
]
#search=[['incubation','period','range','mean','%']]
search=[['incubation','period','range'],
['viral','shedding','duration'],
['asymptomatic','shedding'],
['incubation','period','age','statistically','significant'],
['incubation','period','child'],
['incubation','groups','risk'],
['basic','reproduction', 'number','%'],
['asymptomatic','infection','%'],
['asymptomatic','children'],
['seasonal','transmission'],
['contaminat','object'],
['environmental', 'conditions'],
['sputum','stool','blood','urine'],
['persistence','surfaces'],
['duration','viral','shedding'],
['shedding','stool'],
['shedding','nasopharynx'],
['shedding','blood'],
['shedding','urine'],
['diagnostics','point'],
['model', 'disease'],        
['model', 'infection'],
['model', 'transmission'],
['phenotypic'],
['mutation'],
['immune','response'],
['restriction', 'movement'],
['protective','clothing'],
['transmission','routes']
]
q=0
for search_words in search:
    str1=''
    # a make a string of the search words to print readable version above table
    str1=' '.join(questions[q])
    
    #search the dataframe for all words
    df1=search_dataframe(df,search_words)

    # analyze search results for relevance 
    df1=search_relevance(df1,search_words)

    # get best sentences
    df_table=get_sentences(df1,search_words)
    
    length=df_table.shape[0]
    #limit 3 results
    df_table=df_table.head(2)
    df_table=df_table.drop(['rel_score'], axis=1)
    #convert df to html
    df_table=HTML(df_table.to_html(escape=False,index=False))
    
    # display search topic
    display(HTML('<h3>'+str1+'</h3>'))
    
    #display table
    if length<1:
        print ("No reliable answer could be located in the literature")
    else:
        display(df_table)
    q=q+1
print ('done')

pub_date,authors,title,excerpt
2020-03-18,Xue et al.,Is a 14-day quarantine period optimal for effectively controlling coronavirus disease 2019 (COVID-19)?,Results the full range of incubation periods of the covid-19 cases ranged from 0 to 33 days among 2015 cases.
2020-03-08,Wei et al.,Transmission of corona virus disease 2019 during the incubation period may lead to a quarantine loophole,"Results: the estimated mean incubation period for covid-19 was 4.9 days (95% confidence interval [ci], 4.4 to 5.4) days, ranging from 0.8 to 11.1 days (2.5th to 97.5th percentile)."


pub_date,authors,title,excerpt
2018-11-01,"Eichenberger, et al.","Incidence, significance, and persistence of human coronavirus infection in hematopoietic stem cell transplant recipients","Twenty-seven (59%) patients who underwent repeat np swab had persistent viral shedding for ≥21 days, with a median duration of 4 weeks of viral shedding."
2020-03-11,"Zhou, et al.","Clinical course and risk factors for mortality of adult inpatients with COVID-19 in Wuhan, China: a retrospective cohort study","Median duration of viral shedding was 20·0 days (iqr 17·0–24·0) in survivors, but sars-cov-2 was detectable until death in non-survivors.The longest observed duration of viral shedding in survivors was 37 days."


pub_date,authors,title,excerpt
2020-01-01,"Fung, et al.",A tug-of-war between severe acute respiratory syndrome coronavirus 2 and host antiviral defence: lessons from other pathogenic viruses,"Important questions concerning the interaction between sars-cov-2 and host antiviral defence, including asymptomatic and presymptomatic virus shedding, are also discussed."
2020-03-16,"Yuen, et al.",SARS-CoV-2 and COVID-19: The most important research questions,"Here we highlight nine most important research questions concerning virus transmission, asymptomatic and presymptomatic virus shedding, diagnosis, treatment, vaccine development, origin of virus and viral pathogenesis."


pub_date,authors,title,excerpt
2020-02-29,Henry et al.,Estimate the incubation period of coronavirus 2019 (COVID-19),We found that the incubation periods of the groups with age>=40 years and age<40 years demonstrated a statistically significant difference.


pub_date,authors,title,excerpt
2020-03-18,Xue et al.,Is a 14-day quarantine period optimal for effectively controlling coronavirus disease 2019 (COVID-19)?,The median incubation period of both male and female adults was similar (7-day) but significantly shorter than that (9-day) of child cases (p=0.02).


...

pub_date,authors,title,excerpt
2020-02-20,Alexandra et al.,"Structure, function and antigenicity of the SARS-CoV-2 spike glycoprotein",Coronavirus spike (s) glycoprotein trimers promote entry into cells and are the main target of the humoral immune response.
2018-04-04,"Beidas, et al.",PCR array profiling of antiviral genes in human embryonic kidney cells expressing human coronavirus OC43 structural and accessory proteins,Both sars-cov and mers-cov were shown to express proteins with the potential to evade early innate immune responses.


pub_date,authors,title,excerpt
2020,"Nishiura, et al.",Backcalculating the Incidence of Infection with COVID-19 on the Diamond Princess,The estimated number of new infections among passengers without close contact was very small from 5 february on which a movement restriction policy was imposed.Movement restriction greatly reduced the number of infections from 5 february onwards.
2020-06-01,"Mizumoto, et al.","Estimating Risk for Death from 2019 Novel Coronavirus Disease, China, January-February 2020","The elevated death risk estimates are probably associated with a breakdown of the healthcare system, indicating that enhanced public health interventions, including social distancing and movement restrictions, should be implemented to bring the covid-19 epidemic under control."


pub_date,authors,title,excerpt
2020-03-03,"Mahase, et al.",Covid-19: UK could delay non-urgent care and call doctors back from leave and retirement,"Protective clothing),” the document said.however, the plan did not include details of how or when these measures would be …."
2020-03-05,"Xiao, et al.",Taking the right measures to control COVID-19,"High-filtration masks such as n95 masks and protective clothing (goggles and gowns) should be used in hospitals where health-care workers are in direct contact with infected patients.5 third, the practice of blocking traffic and lockdown of villages is of no value for the prevention and control of covid-19."


pub_date,authors,title,excerpt
2020-02-27,Pengfei et al.,"Clinical features and sexual transmission potential of SARS-CoV-2 infected female patients: a descriptive study in Wuhan, China","Thus, in order to effectively block its transmission, it is urgent to uncover all the possible transmission routes of sars-cov-2."
2020,"Guan, et al.",Epidemiological investigation of a family clustering of COVID-19,"Routes of transmission include respiratory droplets (from case 1 transmitted to case 6, from case 1 to her family members) and closecontact (from case 1 to other cases in her family)."


done
